### 1.Load Data

In [1]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [2]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


In [3]:
import time

start = time.time()
df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')
end = time.time()
print "Time Used %fs" %(end - start)

Time Used 89.718739s


### 2.Data Exploration

In [15]:
print df.columns

print df['reviewText'][0]

df.head(5)

Index([u'reviewerID', u'asin', u'reviewerName', u'helpful', u'unixReviewTime',
       u'reviewText', u'overall', u'reviewTime', u'summary'],
      dtype='object')
This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++


,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",1297468800,This is a great tutu and at a really great pri...,5.0,"02 12, 2011",Great tutu- not cheaply made
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",1358553600,I bought this for my 4 yr old daughter for dan...,5.0,"01 19, 2013",Very Cute!!
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",1357257600,What can I say... my daughters have it in oran...,5.0,"01 4, 2013",I have buy more than one
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]",1398556800,"We bought several tutus at once, and they are ...",5.0,"04 27, 2014","Adorable, Sturdy"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",1394841600,Thank you Halo Heaven great product for Little...,5.0,"03 15, 2014",Grammy's Angels Love it


In [5]:
len(df)

278677

In [6]:
import numpy as np
helpfulness = []
for i in df['helpful']:
    if i[1] == 0:
        helpfulness.append(np.nan)
    else:
        helpfulness.append(float(i[0])/i[1])

In [7]:
sum(np.isnan(helpfulness[0:100]))
print df['helpful'][0:30]

0     [0, 0]
1     [0, 0]
2     [0, 0]
3     [0, 0]
4     [0, 0]
5     [0, 0]
6     [0, 0]
7     [0, 0]
8     [0, 0]
9     [7, 8]
10    [0, 0]
11    [0, 0]
12    [0, 0]
13    [1, 1]
14    [0, 0]
15    [0, 0]
16    [0, 0]
17    [0, 0]
18    [0, 0]
19    [0, 0]
20    [0, 0]
21    [0, 0]
22    [1, 2]
23    [1, 1]
24    [0, 1]
25    [1, 2]
26    [0, 0]
27    [0, 0]
28    [0, 0]
29    [0, 0]
Name: helpful, dtype: object


In [8]:
print "average helpfulness %f" %np.nanmean(helpfulness)
print "number of reviews with helpfulness score: %i" %np.count_nonzero(~np.isnan(helpfulness))

average helpfulness 0.780014
number of reviews with helpfulness score: 87021


### 3.Build Baseline Model

In [9]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection u'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/legu/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/legu/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/legu/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /Users/legu/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/legu/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/legu/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to
[nltk_data]    |     /Users/legu/nltk_data...
[nltk_d

True

In [23]:
#extract the review text
doc = df['reviewText']

In [18]:
import nltk
import string

from collections import Counter

def get_tokens(docs):
    tokens =[]
    for i in docs:
        lowers = i.lower()
        #remove the punctuation using the character deletion step of translate
        no_punctuation = lowers.translate(None, string.punctuation)
        token = nltk.word_tokenize(no_punctuation)
        tokens.extend(token)
    return tokens

In [29]:
##only use the first 10000 doc, since it is too large
token = get_tokens(doc[0:10000])

In [33]:
from nltk.corpus import stopwords
print len( token)
from collections import Counter
filtered = [w for w in token if not w in stopwords.words('english')]
count = Counter(filtered)
print count.most_common(10)

628545
[('fit', 3336), ('like', 3278), ('size', 2917), ('wear', 2881), ('great', 2782), ('good', 2526), ('well', 2480), ('one', 2245), ('comfortable', 2111), ('would', 2076)]


In [38]:
len(count)

22233

In [39]:
##tf-idf transformation
##based on this page: http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
import string
from sklearn.feature_extraction.text import TfidfVectorizer
token_dict = {}

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

for i, text in enumerate(doc[0:10000]):
    
    lowers = text.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[i] = no_punctuation
        
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

In [61]:

print tfs.shape
print tfs[0,:] #sparse matrix
print token_dict[0]
print  tfidf.vocabulary_['tutu']

(10000, 22073)
  (0, 14655)	0.317550910317
  (0, 10522)	0.211873732341
  (0, 1841)	0.267801046641
  (0, 2076)	0.170978738463
  (0, 11668)	0.210168766801
  (0, 8885)	0.219461628674
  (0, 10055)	0.139858612687
  (0, 4340)	0.20913646907
  (0, 11662)	0.136035099096
  (0, 6602)	0.179402243162
  (0, 14862)	0.139170529592
  (0, 15496)	0.135552487933
  (0, 20186)	0.68078334396
  (0, 9123)	0.213979934416
this is a great tutu and at a really great price it doesnt look cheap at all im so glad i looked on amazon and found such an affordable tutu that isnt made poorly a
20186


In [75]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /Users/legu/anaconda/lib/python2.7/site-packages


In [96]:
# helpfulness[0:10000].reshape(1000,1)
y = np.reshape(helpfulness[0:10000],(10000,-1))

In [111]:
nan_ind = (np.isnan(helpfulness[0:10000]))
tfs_remove_nan = tfs[~nan_ind]
y_remove_nan = y[~nan_ind]

In [115]:
#random split
msk = np.random.rand(tfs_remove_nan.shape[0]) <= 0.7

X_train = tfs_remove_nan[msk]
X_test = tfs_remove_nan[~msk]
y_train = y_remove_nan[msk]
y_test = y_remove_nan[~msk]

In [116]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(2072, 22073)
(887, 22073)
(2072, 1)
(887, 1)


In [136]:
#build linear model
from sklearn import datasets, linear_model
from sklearn import metrics
# import matplotlib.pyplot as plt
# load the iris datasets

# fit a logistic regression model to the data
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)
print(model)
# # make predictions
# expected = dataset.target
# predicted = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
('Coefficients: \n', array([[-0.1934025 ,  0.        , -0.06448608, ...,  0.07231034,
         0.05332772,  0.18668477]]))
Mean squared error: 0.17
Variance score: -0.59


In [133]:
print predicted.shape
print y_test.shape

(887, 1)
(887, 1)


## Scratch Below

In [ ]:
print 1

In [62]:
import nltk
nltk.download('brown')

corpus = nltk.corpus.brown

[nltk_data] Downloading package brown to /Users/legu/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [72]:
!pip install vocabulary

    100% |████████████████████████████████| 593kB 408kB/s 
    100% |████████████████████████████████| 61kB 1.9MB/s 
    100% |████████████████████████████████| 102kB 1.2MB/s 
  Running setup.py bdist_wheel for vocabulary ... - \ done
  Stored in directory: /Users/legu/Library/Caches/pip/wheels/36/6c/c0/92bb20f79402d055c3bce3e89d9f2cce5d6937bc2aadc0fb45
Successfully built vocabulary
  Found existing installation: requests 2.11.1
    Uninstalling requests-2.11.1:
      Successfully uninstalled requests-2.11.1
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [73]:
import utils
import vocabulary
# train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=True)
vocab = vocabulary.Vocabulary(utils.canonicalize_word(w) for w in utils.flatten(corpus))
print "Train set vocabulary: %d words" % vocab.size

AttributeError: 'module' object has no attribute 'Vocabulary'